In [1]:
import pynwb

import numpy as np

from pynwb import NWBHDF5IO, NWBFile, TimeSeries

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import bisect
import ssm

import autograd.numpy as np
import autograd.numpy.random as npr


from data_loaders import *

In [5]:
# Extract Date
filename = "sub-monk-g_ses-session0.nwb"
# filename = "sub-monk-g_ses-session2.nwb"
# filename = "sub-monk-g_ses-session3.nwb"
# filename = "sub-monk-j_ses-session1.nwb"
acqRate=60
newRate=60
num_clusters = 8
with NWBHDF5IO(filename , "r") as io:
    read_nwbfile = io.read()    
    trial_num = len(read_nwbfile.trials["id"].data)
    lastTrialId = getLastTrialId(read_nwbfile, acqRate=acqRate, newRate=newRate)
    cursor_position_list=CursorPositionMatrix_list(read_nwbfile, 
                                                   lastTrialId, acqRate=acqRate, newRate = newRate)
    last_cursor =[]
    for cursor_position in cursor_position_list:
        last_cursor.append(cursor_position[-1,:])
    # Get the stop index(in no intertrial data) of every trial
    stop_Indices=[]
    accumulated_idx=0
    for cursor_position in cursor_position_list:
        accumulated_idx += len(cursor_position)
        stop_Indices.append(accumulated_idx-1)

/opt/miniconda3/envs/cse446/lib/python3.9/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/opt/miniconda3/envs/cse446/lib/python3.9/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.5.0 because version 2.6.0-alpha is already loaded.
  return func(args[0], **pargs)
/opt/miniconda3/envs/cse446/lib/python3.9/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)


In [6]:
# Fit LDS
num_permutation = 5
original_input, original_spike, p_input_matrices, p_spike_matrices = getData_for_LDS(filename, 
                                                                                   num_permutation = num_permutation, 
                                                                                    acqRate = acqRate, newRate = newRate,
                                                                                    num_clusters = num_clusters)
obs=original_spike
inputs = original_input

# Known paramters
time_bins = obs.shape[0]
obs_dim = obs.shape[1]
input_dim = 4

# Assumption
state_dim = 4

# fit the model
# Poisson emission
lds_inp = ssm.LDS(obs_dim, state_dim, M=input_dim, emissions="poisson")
elbos, q = lds_inp.fit((obs).astype(int), inputs=inputs, num_iters=20)
state_means = q.mean_continuous_states[0]

/opt/miniconda3/envs/cse446/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[array([[-6.4173536 , -6.79019157, -6.96700622, -6.23898679],
       [-6.4173536 , -6.79019157, -6.96700622, -6.23898679],
       [-6.4173536 , -6.79019157, -6.85184414, -7.67753436],
       ...,
       [-6.4173536 , -6.79019157, -0.62376637,  0.35468384],
       [-6.4173536 , -6.79019157, -0.71642252,  0.50841349],
       [-6.4173536 , -6.79019157, -0.71642252,  0.50841349]]), array([[ 0.20705055, -9.78660199, -0.87772279, -9.35717099],
       [ 0.20705055, -9.78660199, -0.87772279, -9.35717099],
       [ 0.20705055, -9.78660199, -0.87772279, -9.35717099],
       ...,
       [ 0.20705055, -9.78660199, -0.09349443, -0.33880989],
       [ 0.20705055, -9.78660199, -0.34932736, -0.28625089],
       [ 0.20705055, -9.78660199, -0.34932736, -0.28625089]]), array([[ 7.44886441,  5.99805499,  6.77401226,  6.65425931],
       [ 7.44886441,  5.99805499,  6.77401226,  6.65425931],
       [ 7.44886441,  5.99805499,  6.77401226,  6.65425931],
       [ 7.44886441,  5.99805499,  6.77401226,  6.654259

AttributeError: module 'ssm' has no attribute 'LDS'

In [ ]:
# Eigenvalues of A
A = lds_inp.dynamics.A
b = lds_inp.dynamics.b
B = lds_inp.dynamics.Vs
eigenvalues, eigenvectors = np.linalg.eig(A)
print('eigenvalues', eigenvalues)

In [ ]:
# target position
colors = ['blue', 'black','red','green','m','orange', 'gray','c']
cluster_assignments, center_xs, center_ys = Cluster(last_cursor, 
                                                    num_clusters=num_clusters, 
                                                    acqRate = acqRate, 
                                                    vis = False)
print(center_xs)
for i, center in enumerate(center_xs):
    plt.plot(center, center_ys[i], 'o', color=colors[i], label = f'center {i+1}')
plt.title('Target Position', fontsize = 20)
plt.legend()

$x_t = A x_{t-1} + V u_{t} + b + w_t$

At fixed point $x^*$,

$x^* = Ax^* + Vu_{t} + b + w_t$

$(I-A)x^* = Vu_{t} + b + w_t$

In [ ]:
# Get analytical fixed points of latent dynamics
FPs=[]
for i in range(len(center_xs)):
    center = np.array([center_xs[i], center_ys[i]])
    inputs = np.concatenate([center, [0, 0]])
    inputs = inputs.reshape(-1, 1)
    x = np.linalg.solve(np.eye(A.shape[0])-A, (B @ inputs).reshape(state_dim,1) + b.reshape(state_dim,1))
    FPs.append(x.reshape(1,-1))

FPs = np.array(FPs).reshape(len(center_xs), state_dim)  
print('fixed points\n', FPs)

# Visualize 4d fixed points on top 2 PCs
pca = PCA(n_components=2)
pca.fit(FPs)
FP_2d = pca.transform(FPs)
print('2d fixed points\n',FP_2d)
for i in range(len(FP_2d)):
    plt.scatter(FP_2d[i,0], FP_2d[i,1], s=200, marker = 'X', color = colors[i])
plt.title('Fixed Points', fontsize=15)
plt.xlabel('PC1', fontsize=15)
plt.ylabel('PC2', fontsize=15)

In [ ]:
# fixed points
ls_2d = []
for i in range(len(FP_2d)):
    plt.scatter(FP_2d[i,0], FP_2d[i,1], s=200, marker = 'X', color = colors[i])

# Latent states at the end of each trial
for trialid, stop_idx in enumerate(stop_Indices):
    cps = state_means[stop_idx:stop_idx+1,:]
    points_2d = pca.transform(cps)
    plt.plot(points_2d[:,0], points_2d[:,1], 'o', color = colors[cluster_assignments[trialid]])
    ls_2d.append(points_2d[0])
ls_2d = np.array(ls_2d)
plt.title('Fixed Points and Latent States', fontsize=15)
plt.xlabel('PC1', fontsize=15)
plt.ylabel('PC2', fontsize=15)